# Looking into the details


In [61]:
#import paskages 
import tensorflow as tf
from tensorflow.keras.datasets import imdb
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

import io


In [62]:
print(tf.__version__)

2.13.0


In [63]:
#Loading data 
imdb , info = tfds.load("imdb_reviews" , with_info=True  ,as_supervised=True)

In [64]:
#Get train and test  data 
train_data ,test_data = imdb['train'] , imdb['test']

In [65]:
# Initialize sentences and labels lists
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

TypeError: '_PrefetchDataset' object is not subscriptable

In [66]:
# Loop over all training examples and save the sentences and labels
for s,l in train_data:
    training_sentences.append(s.numpy().decode("utf8"))
    training_labels.append(l.numpy())

In [67]:
# Loop over all training examples and save the sentences and labels
for s,l in test_data:
    testing_sentences.append(s.numpy().decode("utf8"))
    testing_labels.append(l.numpy())

In [25]:
# Convert labels lists to numpy array
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [26]:
vocab_size = 10000
max_length = 120
embedding_dim = 16
trunc_type='post'
oov_tok = "<OOV>"

In [28]:
tokenizer = Tokenizer(num_words=vocab_size ,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

In [29]:
word_index= tokenizer.word_index

In [30]:
sequences = tokenizer.texts_to_sequences(training_sentences)

In [31]:
padded = pad_sequences(sequences , maxlen=max_length , truncating=trunc_type)

In [35]:
# Generate and pad the test sequences
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length, truncating=trunc_type)

In [ ]:
vocab_size = 10000
max_length = 120
embedding_dim = 16
trunc_type='post'
oov_tok = "<OOV>"

In [43]:
#Create model Sequential
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000 , 16 , input_length=120  ),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6 , activation='relu'),
    tf.keras.layers.Dense(1 , activation='sigmoid')
    
])

# Setup the training parameters
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 120, 16)           160000    
                                                                 
 flatten_4 (Flatten)         (None, 1920)              0         
                                                                 
 dense_8 (Dense)             (None, 6)                 11526     
                                                                 
 dense_9 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171533 (670.05 KB)
Trainable params: 171533 (670.05 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [45]:
model.fit(padded ,
         training_labels_final ,
         epochs=10 ,
         validation_data=(testing_padded , testing_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 4ms/step - loss: 0.4975 - accuracy: 0.7408 - val_loss: 0.3863 - val_accuracy: 0.8240
Epoch 2/10
782/782 [==============================] - 3s 4ms/step - loss: 0.2372 - accuracy: 0.9096 - val_loss: 0.4125 - val_accuracy: 0.8190
Epoch 3/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0861 - accuracy: 0.9781 - val_loss: 0.5178 - val_accuracy: 0.8061
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0217 - accuracy: 0.9971 - val_loss: 0.6164 - val_accuracy: 0.8037
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0053 - accuracy: 0.9997 - val_loss: 0.7000 - val_accuracy: 0.8032
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.7461 - val_accuracy: 0.8077
Epoch 7/10
782/782 [==============================] - 4s 6ms/step - loss: 8.5100e-04 - accuracy: 1.0000 - val_loss: 0.7941 - val_accuracy: 0.8084
Ep

In [58]:
embedding_layer = model.layers[0]

embedding_weights = embedding_layer.get_weights()[0]

print(embedding_weights.shape) 

(10000, 16)


In [48]:
# Get the index-word dictionary
reverse_word_index = tokenizer.index_word

In [52]:
# Open writeable files
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

In [59]:
for word_num in range(1, vocab_size):

  # Get the word associated at the current index
  word_name = reverse_word_index[word_num]

  # Get the embedding weights associated with the current index
  word_embedding = embedding_weights[word_num]

  # Write the word name
  out_m.write(word_name + "\n")

  # Write the word embedding
  out_v.write('\t'.join([str(x) for x in word_embedding]) + "\n")

# Close the files
out_v.close()
out_m.close()

In [60]:
# Import files utilities in Colab
try:
    from google.colab import files
except ImportError:
    pass

# Download the files
else:
    files.download('vecs.tsv')
    files.download('meta.tsv')